# Prediction Report & Data Enhancement
### Braden Poe | Nationwide Work Sample

## Overview

Using Las Vegas restaurant inspection data we want to predict the probability of a restaurant receiving a critical inspection. The current trimmed dataset identifies restaurants by city, zipcode, lagged inspection data, and a binary response variable. Roughly 15,000 observations are available through which the model can be constructed. Two models were seriously considered in this portion of the analysis: Random Forest, and Logistic Regression. 

I ruled out logistic regression due to the inseparability of the data and high dimensionality of the dataset. Speaking to inseparability, I felt that any type of linear boundary would do a poor job at classification. This concern could be addressed with a non-linear boundary, but the already high dimensionality of the dataset would lead to a massive polynomial in the loss function. I could have lessened dimensionality through feature selection, but that was not the most efficient option with other techniques on the table. 

Random forest seemed like the most effective method for dealing with inseparability; given the nice distributional spread across "lagged demerits" and "lagged violations" I am confident that no extreme outliers exist. Estimation can be poor for rare observations if the dataset is not representative of the population, but since the data featured every Las Vegas restaurant, that was not a concern. Further, the ability to tune hyperparameters was a major advantage. With logistic regression I would have implemented an L1 penalty to more aggressively eliminate features, but that is the only hyperparameter I could have tuned. Random forest classification opens up the possibility for significant model adjustments that can effectively mimic the non-linearity required by the data.

As a final consideration, I briefly looked at implementing a neural net, but that seemed like overkill given the sample size. If I was working with a national dataset or even a statewide dataset, then a neural net may have been a more logical choice.

---

## Import Data & Baseline

First, I want to import the final csv from the previous report and then establish a naive baseline to which I can compare my trained model. The naive baseline will simply be predicting one every time, since 'Response' = 1 is the majority of our data.

In [7]:
# Import model
import numpy as np
import pandas as pd

#Import data
data = pd.read_csv('predict.csv',index_col=None)
data = data.drop('Unnamed: 0',axis=1)
data.insert(0,'index',range(0,len(data)))
data = data.set_index('index')

# Set X and y data 
y = data['response']
X = data.drop('response',axis=1)
col_names  = X.columns
col = np.array(col_names).reshape(127,1)
print('The naive prediction accuracy when predicting response equal to 1 is the mean of our response variable.')
print('\nNaive Accuracy: {0}'.format(round(y.mean(),4)))

The naive prediction accuracy when predicting response equal to 1 is the mean of our response variable.

Naive Accuracy: 0.5768


Ignoring precision and recall for the moment, if the trained model accuracy cannot beat the naive prediction accuracy above then there is no way to justify implementation. I hope to see accuracy over 90%, but am keeping in mind that many demographic variables are missing from the dataset.

---

## Feature Selection: Random Forest

#### Model Baseline without Feature Selection, Tuning, or CV

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split as tts
import warnings
import timeit

t0 = timeit.default_timer()
warnings.filterwarnings('ignore')

# Train and test splits
X_train, X_test, y_train, y_test = tts(X,y, test_size=.3, random_state= 1)

# Let's run a straight forward estimate to see how the model generally does w/out tuning or CV

rf = RandomForestClassifier(n_estimators=1000, max_depth = 50,random_state=42)
rf.fit(X_train, y_train)
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
t20 = feature_importances['importance'].head(20)
pred = rf.predict(X_test)
pred[pred >= 0.5] = 1
pred[pred < 0.5] = 0
pred = pred.astype(int)


# Confusion Matrix
cm = pd.crosstab(y_test,pred,rownames=['Actual'],colnames=['Predicted'])
print(cm)

# Accuracy, Recall, Precision
pre = cm.loc[1,1]/(cm.loc[0,1]+cm.loc[1,1])
print('\nThe model precision is: {0}'.format(round(pre,3)))
rec = cm.loc[1,1]/(cm.loc[1,0]+cm.loc[1,1])
print('The model recall is: {0}'.format(round(rec,3)))
acc = (cm.loc[1,1]+cm.loc[0,0])/(cm.loc[1,1]+cm.loc[0,0] + cm.loc[0,1] + cm.loc[1,0])
print('The model accuracy is: {0}'.format(round(acc,3)))
t1 = timeit.default_timer()
total = t1 - t0
print('\n\nTook {0} seconds to run.'.format(round(total,2)))

Predicted     0     1
Actual               
0          1021   998
1           821  1881

The model precision is: 0.653
The model recall is: 0.696
The model accuracy is: 0.615


Took 44.27 seconds to run.


This model with no tuning is only 4% more accurate than the naive baseline at 62% with precision and recall at roughly 70%. The early returns are not ideal - I would have liked to see an accuracy score in the mid 70's with recall correspondingly high since we would be worried about false negatives.

#### Feature Selection

I am taking the 20 most important features from the previously trained model and will manually narrow that list down further to gain an optimal number of features.

In [9]:
# Adjust X and y to reflect top 20 features
cols_to_keep = list(t20.index)
X = X[cols_to_keep]
y = y

# New Baseline
t0 = timeit.default_timer()
warnings.filterwarnings('ignore')

# Train and test splits
X_train, X_test, y_train, y_test = tts(X,y, test_size=.3, random_state= 1)

# Let's run a straight forward estimate to see how the model generally does w/out tuning or CV

rf = RandomForestClassifier(n_estimators=1000, max_depth = 50,random_state=42)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)
pred[pred >= 0.5] = 1
pred[pred < 0.5] = 0
pred = pred.astype(int)
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
t10 = feature_importances['importance'].head(10)

# Confusion Matrix
cm = pd.crosstab(y_test,pred,rownames=['Actual'],colnames=['Predicted'])
print(cm)

# Accuracy, Recall, Precision
pre = cm.loc[1,1]/(cm.loc[0,1]+cm.loc[1,1])
print('\nThe model precision is: {0}'.format(round(pre,3)))
rec = cm.loc[1,1]/(cm.loc[1,0]+cm.loc[1,1])
print('The model recall is: {0}'.format(round(rec,3)))
acc = (cm.loc[1,1]+cm.loc[0,0])/(cm.loc[1,1]+cm.loc[0,0] + cm.loc[0,1] + cm.loc[1,0])
print('The model accuracy is: {0}'.format(round(acc,3)))
t1 = timeit.default_timer()
total = t1 - t0
print('\n\nTook {0} seconds to run.'.format(round(total,2)))

Predicted     0     1
Actual               
0          1021   998
1           733  1969

The model precision is: 0.664
The model recall is: 0.729
The model accuracy is: 0.633


Took 17.11 seconds to run.


Accuracy and precision improved slightly with narrower features. I will consider narrowing to 10 features to see if results change at all. In the interest of computation time, the relevant features should be minimized while maximizing accuracy.

In [10]:
# Adjust X and y to reflect top 10 features
cols_to_keep = list(t10.index)
X = X[cols_to_keep]
y = y

# New Baseline
t0 = timeit.default_timer()
warnings.filterwarnings('ignore')

# Train and test splits
X_train, X_test, y_train, y_test = tts(X,y, test_size=.3, random_state= 1)

# Let's run a straight forward estimate to see how the model generally does w/out tuning or CV

rf = RandomForestClassifier(n_estimators=1000, max_depth = 50,random_state=42)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)
pred[pred >= 0.5] = 1
pred[pred < 0.5] = 0
pred = pred.astype(int)
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
t5 = feature_importances['importance'].head(5)

# Confusion Matrix
cm = pd.crosstab(y_test,pred,rownames=['Actual'],colnames=['Predicted'])
print(cm)

# Accuracy, Recall, Precision
pre = cm.loc[1,1]/(cm.loc[0,1]+cm.loc[1,1])
print('\nThe model precision is: {0}'.format(round(pre,3)))
rec = cm.loc[1,1]/(cm.loc[1,0]+cm.loc[1,1])
print('The model recall is: {0}'.format(round(rec,3)))
acc = (cm.loc[1,1]+cm.loc[0,0])/(cm.loc[1,1]+cm.loc[0,0] + cm.loc[0,1] + cm.loc[1,0])
print('The model accuracy is: {0}'.format(round(acc,3)))
t1 = timeit.default_timer()
total = t1 - t0
print('\n\nTook {0} seconds to run.'.format(round(total,2)))

Predicted     0     1
Actual               
0          1049   970
1           736  1966

The model precision is: 0.67
The model recall is: 0.728
The model accuracy is: 0.639


Took 12.81 seconds to run.


In [11]:
# Adjust X and y to reflect top 5 features
cols_to_keep = list(t5.index)
X = X[cols_to_keep]
y = y

# New Baseline
t0 = timeit.default_timer()
warnings.filterwarnings('ignore')

# Train and test splits
X_train, X_test, y_train, y_test = tts(X,y, test_size=.3, random_state= 1)

# Let's run a straight forward estimate to see how the model generally does w/out tuning or CV

rf = RandomForestClassifier(n_estimators=1000, max_depth = 50,random_state=42)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)
pred[pred >= 0.5] = 1
pred[pred < 0.5] = 0
pred = pred.astype(int)
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
t4 = feature_importances['importance'].head(4)

# Confusion Matrix
cm = pd.crosstab(y_test,pred,rownames=['Actual'],colnames=['Predicted'])
print(cm)

# Accuracy, Recall, Precision
pre = cm.loc[1,1]/(cm.loc[0,1]+cm.loc[1,1])
print('\nThe model precision is: {0}'.format(round(pre,3)))
rec = cm.loc[1,1]/(cm.loc[1,0]+cm.loc[1,1])
print('The model recall is: {0}'.format(round(rec,3)))
acc = (cm.loc[1,1]+cm.loc[0,0])/(cm.loc[1,1]+cm.loc[0,0] + cm.loc[0,1] + cm.loc[1,0])
print('The model accuracy is: {0}'.format(round(acc,3)))
t1 = timeit.default_timer()
total = t1 - t0
print('\n\nTook {0} seconds to run.'.format(round(total,2)))

Predicted     0     1
Actual               
0          1011  1008
1           666  2036

The model precision is: 0.669
The model recall is: 0.754
The model accuracy is: 0.645


Took 12.76 seconds to run.


In [12]:
# Adjust X and y to reflect top 4 features
cols_to_keep = list(t4.index)
X = X[cols_to_keep]
y = y

# New Baseline
t0 = timeit.default_timer()
warnings.filterwarnings('ignore')

# Train and test splits
X_train, X_test, y_train, y_test = tts(X,y, test_size=.3, random_state= 1)

# Let's run a straight forward estimate to see how the model generally does w/out tuning or CV

rf = RandomForestClassifier(n_estimators=1000, max_depth = 50,random_state=42)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)
pred[pred >= 0.5] = 1
pred[pred < 0.5] = 0
pred = pred.astype(int)
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
t4 = feature_importances['importance'].head(4)

# Confusion Matrix
cm = pd.crosstab(y_test,pred,rownames=['Actual'],colnames=['Predicted'])
print(cm)

# Accuracy, Recall, Precision
pre = cm.loc[1,1]/(cm.loc[0,1]+cm.loc[1,1])
print('\nThe model precision is: {0}'.format(round(pre,3)))
rec = cm.loc[1,1]/(cm.loc[1,0]+cm.loc[1,1])
print('The model recall is: {0}'.format(round(rec,3)))
acc = (cm.loc[1,1]+cm.loc[0,0])/(cm.loc[1,1]+cm.loc[0,0] + cm.loc[0,1] + cm.loc[1,0])
print('The model accuracy is: {0}'.format(round(acc,3)))
t1 = timeit.default_timer()
total = t1 - t0
print('\n\nTook {0} seconds to run.'.format(round(total,2)))

Predicted    0     1
Actual              
0          958  1061
1          648  2054

The model precision is: 0.659
The model recall is: 0.76
The model accuracy is: 0.638


Took 9.36 seconds to run.


In [13]:
t4

demerits_lag    0.370284
nm_lag          0.256711
numvio_lag      0.211941
crit_lag        0.161063
Name: importance, dtype: float64

### Feature Selection Conclusion

The process to determine the optimal number of features was a bit roundabout, but I have settled on 4 final features to be included in the model because they maximize model precision, recall, and accuracy. The final features chosen represent lags of non-major violations, demerits, number of violations, and critical violations. There is nothing surprising about these features being the most important. Since there are not substantial levels of collinearity perent, I am also confident that these four features are truly the strongest predictors of critical violations. 

The gain in overall accuracy is only 7% compared to the naive baseline, but hopefully this value can be increased through hyperparameter tuning. It is good that we have higher recall in this scenario, because the city of Las Vegas will want to place a strong emphasis on identifying critical violation restaurants. Recall at 76% means we are catching over 3/4 of critical violators with the model, which is certainly better than the naive baseline.

---

## Hyperparameter Tuning & Final Estimation

#### Tuning

In [14]:
from sklearn.model_selection import RandomizedSearchCV

# Edits to parameters in random forest model
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 1000, stop = 2000, num = 11)]
max_features = ['auto',None]
max_depth = [int(x) for x in np.linspace(1, 51, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [2, 5, 10]
bootstrap = [True, False]

# Create the random grid
params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


X_train, X_test, y_train, y_test = tts(X,y, test_size=.3, random_state= 1)

# Use the random grid to search for best hyperparameters
# Search across 50 iterations and 5 cvs, which totals 250 combos
rf = RandomForestClassifier()
rf_hyper = RandomizedSearchCV(estimator = rf, param_distributions = params, 
                               n_iter = 50, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_hyper.fit(X_train, y_train)

# Display best parameters
rf_hyper.best_params_

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 13.2min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 20.6min finished


{'n_estimators': 1200,
 'min_samples_split': 10,
 'min_samples_leaf': 10,
 'max_features': 'auto',
 'max_depth': 46,
 'bootstrap': True}

#### Final Model Results

In [15]:
# Use best params to get final model efficiency 
t0 = timeit.default_timer()

rf_final = RandomForestClassifier(n_estimators = 1200, min_samples_split = 10, min_samples_leaf = 10,
                                 max_features = 'auto', max_depth = 46, bootstrap = True)
rf_final.fit(X_train, y_train)
pred = rf_final.predict(X_test)
pred[pred >= 0.5] = 1
pred[pred < 0.5] = 0
pred = pred.astype(int)

# Confusion Matrix
cm = pd.crosstab(y_test,pred,rownames=['Actual'],colnames=['Predicted'])
print(cm)

# Accuracy, Recall, Precision
pre = cm.loc[1,1]/(cm.loc[0,1]+cm.loc[1,1])
print('\nThe model precision is: {0}'.format(round(pre,3)))
rec = cm.loc[1,1]/(cm.loc[1,0]+cm.loc[1,1])
print('The model recall is: {0}'.format(round(rec,3)))
acc = (cm.loc[1,1]+cm.loc[0,0])/(cm.loc[1,1]+cm.loc[0,0] + cm.loc[0,1] + cm.loc[1,0])
print('The model accuracy is: {0}'.format(round(acc,3)))
t1 = timeit.default_timer()
total = t1 - t0
print('\n\nTook {0} seconds to run.'.format(round(total,2)))

Predicted    0     1
Actual              
0          951  1068
1          608  2094

The model precision is: 0.662
The model recall is: 0.775
The model accuracy is: 0.645


Took 10.25 seconds to run.


After feature selection and hyperparameter training we acheive a recall of 77.5%, which is an upgrade over the naive baseline prediction accuracy of 57.7%. 

---

## Model Recommendation

The final model is not a minimally viable product in its current state due to a few shortcomings. First, the recall of 77.5% improves upon the naive baseline, but I would not be comfortable selling this model to a third party because of the high false negative rate. Several data points would need to be merged into the final dataset in order to add a higher level of richness. I will discuss these data points in detail in the section below.

In addition to recall concerns, adding lags into data significantly impacted the scope of restaurants to which we could apply the model. To reiterate, lags were added as main features because an employee working for the City of Las Vegas would only know the previous performance of a restaurant before completing another inspection. Not lagging the data creates a model where the employee is trying to predict a critical violation in time 't', but needs to enter restaurant characterstics from time 't'. The purpose of creating the model is to make critical violations predictable without needing to go to the actual location first. 

The drawbacks of using lags meant that any restaurant with only one recorded inspection was effectively removed from the dataset. This decision restricted the dataset to ~15,000 observations from ~27,000 observations. So, we can predict critical violations fairly well for restaurants that remain in business over multiple inspections, but have unclear power in predicting violations for new restaurants or those with a single inspection. I would argue that the model will still work for restaurants with one inspection, because their shutdown rates are not out-of-sync with multiple inspection restaurants, which signals some level of homogeneity between the two groups. Violations in new restaurants, on the other hand, are unable to be predicted. I would suggest developing an additional model based on population demographics to address the problem posed by new restaurants.

---

## Data Enhancement (Sky is the limit)

#### Demographic Characteristics

1. Median Income of Consumers at a Restaurant
    1. Restaurants may be more inclined to keep their establishment looking nice for customers with higher incomes, regardless of profit margins. Since consumers with higher incomes are able to travel with more ease, their demand is more elastic than the low-income consumer.

2. Crime Rate 
    1. I have worked on a few other projects with population demographics and crime rate has played a key role. I expect this would be no different with restaurant inspections.

3. Cars per capita in vicinity of restaurant 
    1. Are many consumers in your region able to drive to your establishment?

#### Restaurant Characteristics

1. Lease amount ($)
    1. Did it get more expensive to operate the restaurant on a monthly basis? This may mean there is less money leftover to keep the restaurant violation-free come inspection time.

2. Real-time Costs and Revenue
    1. If we knew that a restaurant was in the red or black prior to the inspection, that would be a clear indication of money available to keep the establishment in good shape.

3. Outstanding Debts
    1. This would be another measure of a restaurant's financial flexibility. Collinearity with the previous variable could be present, but I would argue there significant variation between real-time profit margin and cumulative debts.

4. Interest Rates paid on loans for each restaurant
    1. Did your financial situation just get better or worse? 

5. Minimum Wage (or wages paid by each restaurant) & Wages with tips
    1. Another indicator of restaurant status. I imagine minimum-wage restaurants are more delinquent than counterparts who pay competitive wages with higher levels of tips. 

#### Misc. Characteristics

1. Policy Variables
    1. Restaurant inspection codes are created based on city/state regulations. If a major policy or regulation was implemented or changed in 2013, then that could dramatically affect inspection decisions. A handful of former critical violations being downgraded in the middle of our sample is something we want to control.

2. Larger Span of Inspections 
    1. This one is straightforward. The more data points we have over time, the less the dataset is impacted by lags being dropped.